In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from random import randint

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


In [9]:
class InvalidMove(Exception):
    pass


class GameLogic:
    boards = []
    owners = {}
    addresses = []

    def __init__(self, addresses: list) -> None:
        self.addresses = addresses

    def reset(self):
        self.boards = []
        self.owners = {}

    def after_move(self):
        pass

    def game_board(self):
        if len(self.boards) > 64:
            raise Exception("Not implemented - too many boards")

        rows = []

        for i, board in enumerate(self.boards):
            print(np.asarray(rows).shape)
            if i % 8 == 0:
                rows.append(board)
            else:
                rows[i // 8] = np.concatenate((rows[i // 8], board), axis=1)

        print(rows)
        return np.concatenate(rows, axis=0)

    def expand(self, address: str):
        board_id = len(self.boards)
        board = np.full((8, 8), 0)

        self.boards.append(board)
        self.owners[board_id] = address

        return self.boards

    def get_tile(self, board_id, y, x):
        if y < 0 or y > 7:
            return None

        if board_id >= len(self.boards):
            raise Exception("Board not found")

        if x > 7:
            board_id += 1
            x = 0

        elif x < 0:
            board_id -= 1
            x = 7

        if board_id < 0 or board_id >= len(self.boards):
            return None

        account = self.boards[board_id][y][x]

        if account == 0:
            return None

        return {
            "account": account,
            "owner": self.owners[board_id],
            "board": board_id,
            "y": y,
            "x": x,
        }

    def capture(self, tile):
        board_id = tile["board"]
        y = tile["y"]
        x = tile["x"]

        left = self.get_tile(board_id, y, x - 1)
        right = self.get_tile(board_id, y, x + 1)
        above = self.get_tile(board_id, y - 1, x)
        below = self.get_tile(board_id, y + 1, x)

        if left and right and above and below:
            self.boards[board_id][y][x] = 0

    def place(self, address: str, moves: list):
        address_map = {a: i + 1 for i, a in enumerate(self.addresses)}
        account_id = address_map[address]

        for board_id, y, x in moves:
            tile = self.get_tile(board_id, y, x)

            # open tiles only
            if tile is not None:
                raise InvalidMove("Tile is occupied")

            is_owner = self.owners[board_id] == address
            is_edge = x == 0 or x == 7 or y == 0 or y == 7

            left = self.get_tile(board_id, y, x - 1)
            right = self.get_tile(board_id, y, x + 1)
            above = self.get_tile(board_id, y - 1, x)
            below = self.get_tile(board_id, y + 1, x)

            # make the move
            def _move():
                self.boards[board_id][y][x] = account_id

            # the owner can always place on their edges
            if is_owner and is_edge:
                _move()

            elif left and left["account"] == account_id:
                _move()

            elif right and right["account"] == account_id:
                _move()

            elif above and above["account"] == account_id:
                _move()

            elif below and below["account"] == account_id:
                _move()

            else:
                raise InvalidMove("Invalid move")

            tile = self.get_tile(board_id, y, x)
            [self.capture(t) for t in [tile, left, right, above, below] if t]

        self.after_move()

        return self.boards[board_id]


In [10]:
game = GameLogic(["alice", "bob", "john"])
game.reset()
game.expand("alice")
game.expand("bob")
game.expand("alice")
game.expand("bob")
game.expand("alice")
game.expand("bob")
game.expand("alice")
game.expand("bob")
game.expand("alice")
game.expand("bob")
game.expand("alice")
game.expand("bob")
game.render()


AttributeError: 'GameLogic' object has no attribute 'render'

In [ ]:
# Shows capture function
game.reset()

game.expand("alice")
game.place("alice", [[0, 0, 7], [0, 1, 7]])
game.render()

game.expand("bob")
game.place("bob", [[1, 1, 0], [1, 2, 0]])
game.render()

game.place("bob", [[0, 2, 7], [0, 2, 6], [0, 1, 6]])
game.render()


In [ ]:
# Shows capture function
game.reset()

players = ["alice", "bob", "john"]
board_count = 9

for i in range(board_count):
    game.expand(players[i % 3])

for i in range(100):
    for player in players:
        # try placing 100 times
        for _ in range(100):
            move = [randint(0, board_count - 1), randint(0, 7), randint(0, 7)]
            try:
                game.place(player, [move])
                break
            except InvalidMove:
                pass

    if i % 10 == 0:
        game.render()
